In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
from pathlib import Path
from datetime import date

plt.style.use("seaborn-v0_8")

# ============================================================
# SETTINGS
# ============================================================
START_DATE = "2018-01-01"
END_DATE = date.today().strftime("%Y-%m-%d")  # excludes today (yfinance end= is exclusive)

TRADING_DAYS = 252
RISK_FREE_RATE = 0.04

NUM_PORTFOLIOS = 5000
RANDOM_SEED = 42

TICKERS_CSV = "tickers.csv"

# Benchmark options:
# - Australia index: "^AXJO" (ASX 200 index)
# - Australia ETF: "A200.AX"
# - US index: "^GSPC" (S&P 500)
# - US ETF: "SPY"
BENCHMARK_TICKER = "^AXJO"
BENCHMARK_NAME = "Benchmark"

# Power BI export folder
PBI_OUT_DIR = Path("powerbi_exports")


# ============================================================
# FUNCTION: Get company name from Yahoo Finance
# ============================================================
def get_company_name(ticker: str) -> str:
    try:
        info = yf.Ticker(ticker).info
        return info.get("longName") or info.get("shortName") or ticker
    except Exception:
        return ticker


# ============================================================
# PORTFOLIO METRICS (EXPECTED, FROM MEAN/COV)
# ============================================================
def portfolio_performance(weights, mean_returns, cov_matrix, risk_free_rate=RISK_FREE_RATE):
    """
    Expected annualised return, volatility, and Sharpe based on mean_returns and cov_matrix.
    """
    weights = np.array(weights, dtype=float)

    mu = np.array(mean_returns, dtype=float)
    cov = np.array(cov_matrix, dtype=float)

    ret = float(np.dot(weights, mu))
    vol = float(np.sqrt(np.dot(weights.T, np.dot(cov, weights))))
    sharpe = (ret - risk_free_rate) / vol if vol != 0 else 0.0
    return ret, vol, sharpe


# ============================================================
# RANDOM PORTFOLIOS (FOR FRONTIER CLOUD)
# ============================================================
def generate_random_portfolios(num_portfolios, mean_returns, cov_matrix,
                               risk_free_rate=RISK_FREE_RATE, seed=RANDOM_SEED):
    np.random.seed(seed)
    n_assets = len(mean_returns)

    results = {"returns": [], "volatility": [], "sharpe": [], "weights": []}

    for _ in range(num_portfolios):
        weights = np.random.random(n_assets)
        weights /= weights.sum()

        ret, vol, sharpe = portfolio_performance(weights, mean_returns, cov_matrix, risk_free_rate)

        results["returns"].append(ret)
        results["volatility"].append(vol)
        results["sharpe"].append(sharpe)
        results["weights"].append(weights)

    return results


# ============================================================
# BACKTEST + REALISED METRICS (FROM DAILY RETURNS)
# ============================================================
def backtest_portfolio(returns_df: pd.DataFrame, weights: np.ndarray) -> pd.Series:
    """
    returns_df: daily returns DataFrame with columns aligned to tickers
    weights: numpy array aligned to tickers
    returns: daily portfolio returns Series
    """
    w = np.array(weights, dtype=float)
    return (returns_df * w).sum(axis=1)


def perf_metrics_from_daily_returns(daily_ret: pd.Series, risk_free_rate=RISK_FREE_RATE) -> dict:
    """
    Realised metrics from daily returns.
    """
    daily_ret = daily_ret.dropna()
    n = len(daily_ret)
    if n == 0:
        return {"CAGR": np.nan, "Vol": np.nan, "Sharpe": np.nan, "MaxDrawdown": np.nan}

    equity = (1 + daily_ret).cumprod()
    total_return = equity.iloc[-1] - 1

    cagr = (1 + total_return) ** (TRADING_DAYS / n) - 1
    vol = daily_ret.std(ddof=0) * np.sqrt(TRADING_DAYS)
    ann_ret = daily_ret.mean() * TRADING_DAYS
    sharpe = (ann_ret - risk_free_rate) / vol if vol != 0 else np.nan

    running_max = equity.cummax()
    drawdown = equity / running_max - 1
    max_dd = float(drawdown.min())

    return {"CAGR": float(cagr), "Vol": float(vol), "Sharpe": float(sharpe), "MaxDrawdown": max_dd}


# ============================================================
# MAIN
# ============================================================
def main():
    # -------------------------
    # USER INPUT
    # -------------------------
    amount_of_money = float(input("What is the total investment amount: $"))

    tickers_df = pd.read_csv(TICKERS_CSV)
    tickers_df["ticker"] = tickers_df["ticker"].astype(str).str.strip()
    tickers = tickers_df["ticker"].dropna().tolist()

    if len(tickers) == 0:
        raise ValueError("No tickers found in tickers.csv under column 'ticker'.")

    # -------------------------
    # DOWNLOAD PRICE DATA (PORTFOLIO ASSETS)
    # -------------------------
    data = yf.download(
        tickers,
        start=START_DATE,
        end=END_DATE,
        auto_adjust=True,
        progress=True
    )["Close"]

    data = data.ffill().bfill()

    # Only keep tickers that downloaded successfully
    final_tickers = list(data.columns)
    if len(final_tickers) == 0:
        raise ValueError("No price data downloaded. Check tickers or date range.")

    # Company names aligned to final tickers
    company_names = [get_company_name(t) for t in final_tickers]

    print("\nTicker → Company mapping:")
    for t, n in zip(final_tickers, company_names):
        print(f"{t} = {n}")

    latest_prices = data.iloc[-1]  # aligned with final_tickers

    # -------------------------
    # DOWNLOAD BENCHMARK PRICE DATA
    # -------------------------
    bench_close = yf.download(
        BENCHMARK_TICKER,
        start=START_DATE,
        end=END_DATE,
        auto_adjust=True,
        progress=True
    )["Close"]

    # yfinance sometimes returns a 1-col DataFrame; ensure Series
    if isinstance(bench_close, pd.DataFrame):
        bench_close = bench_close.iloc[:, 0]

    bench_close = bench_close.ffill().bfill()

    # -------------------------
    # RETURNS + COVARIANCE
    # -------------------------
    returns = data.pct_change().dropna()
    bench_returns = bench_close.pct_change()

    # Align benchmark returns to portfolio returns dates
    bench_returns = bench_returns.reindex(returns.index).ffill().bfill().dropna()

    # Ensure perfect index match
    common_idx = returns.index.intersection(bench_returns.index)
    returns = returns.loc[common_idx]
    bench_returns = bench_returns.loc[common_idx]

    mean_returns = returns.mean() * TRADING_DAYS
    cov_matrix = returns.cov() * TRADING_DAYS

    # -------------------------
    # RANDOM PORTFOLIOS
    # -------------------------
    rp = generate_random_portfolios(NUM_PORTFOLIOS, mean_returns.values, cov_matrix.values)

    returns_array = np.array(rp["returns"])
    vol_array = np.array(rp["volatility"])
    sharpe_array = np.array(rp["sharpe"])
    weights_array = np.array(rp["weights"])

    # -------------------------
    # OPTIMAL PORTFOLIOS
    # -------------------------
    max_sharpe_idx = sharpe_array.argmax()
    min_vol_idx = vol_array.argmin()

    max_sharpe_w = weights_array[max_sharpe_idx]
    min_vol_w = weights_array[min_vol_idx]

    max_sharpe_ret, max_sharpe_vol, max_sharpe_ratio = portfolio_performance(
        max_sharpe_w, mean_returns.values, cov_matrix.values, RISK_FREE_RATE
    )
    min_vol_ret, min_vol_vol, min_vol_ratio = portfolio_performance(
        min_vol_w, mean_returns.values, cov_matrix.values, RISK_FREE_RATE
    )

    # MaxReturn portfolio (highest expected return among random portfolios)
    max_return_idx = returns_array.argmax()
    max_return_w = weights_array[max_return_idx]
    max_return_ret, max_return_vol, max_return_sharpe = portfolio_performance(
        max_return_w, mean_returns.values, cov_matrix.values, RISK_FREE_RATE
    )

    # -------------------------
    # EQUAL WEIGHT PORTFOLIO
    # -------------------------
    equal_weights = np.ones(len(final_tickers)) / len(final_tickers)
    ew_return, ew_vol, ew_sharpe = portfolio_performance(
        equal_weights, mean_returns.values, cov_matrix.values, RISK_FREE_RATE
    )

    # -------------------------
    # SHARES + DOLLAR ALLOCATIONS
    # -------------------------
    alloc_max_sharpe = max_sharpe_w * amount_of_money
    alloc_min_vol = min_vol_w * amount_of_money
    alloc_equal = equal_weights * amount_of_money
    alloc_max_return = max_return_w * amount_of_money

    shares_max = alloc_max_sharpe / latest_prices.values
    shares_min = alloc_min_vol / latest_prices.values
    shares_eq = alloc_equal / latest_prices.values
    shares_ret = alloc_max_return / latest_prices.values

    # -------------------------
    # CREATE OUTPUT TABLE (BASE, NO TOTAL ROW)
    # -------------------------
    allocations_df_base = pd.DataFrame({
        "Ticker": final_tickers,
        "Company": company_names,
        "LatestPrice": latest_prices.values,

        "MaxSharpeWeight": max_sharpe_w,
        "MaxSharpe$": alloc_max_sharpe,
        "MaxSharpeShares": shares_max,

        "MinVolWeight": min_vol_w,
        "MinVol$": alloc_min_vol,
        "MinVolShares": shares_min,

        "MaxReturnWeight": max_return_w,
        "MaxReturn$": alloc_max_return,
        "MaxReturnShares": shares_ret,

        "EqualWeight": equal_weights,
        "EqualWeight$": alloc_equal,
        "EqualWeightShares": shares_eq
    })

    # TOTAL ROW (for the human-friendly CSV only)
    total_row = pd.DataFrame({
        "Ticker": ["TOTAL"],
        "Company": [""],
        "LatestPrice": [np.nan],

        "MaxSharpeWeight": [max_sharpe_w.sum()],
        "MaxSharpe$": [alloc_max_sharpe.sum()],
        "MaxSharpeShares": [shares_max.sum()],

        "MinVolWeight": [min_vol_w.sum()],
        "MinVol$": [alloc_min_vol.sum()],
        "MinVolShares": [shares_min.sum()],

        "MaxReturnWeight": [max_return_w.sum()],
        "MaxReturn$": [alloc_max_return.sum()],
        "MaxReturnShares": [shares_ret.sum()],

        "EqualWeight": [equal_weights.sum()],
        "EqualWeight$": [alloc_equal.sum()],
        "EqualWeightShares": [shares_eq.sum()]
    })

    allocations_df_output = pd.concat([allocations_df_base, total_row], ignore_index=True)

    # -------------------------
    # EXPORT HUMAN-FRIENDLY CSV
    # -------------------------
    allocations_df_output.to_csv("Portfolio_Optimisation_Results.csv", index=False)
    print("\nCSV saved: Portfolio_Optimisation_Results.csv")

    # =========================================================
    # POWER BI EXPORTS (TIDY TABLES)
    # =========================================================
    PBI_OUT_DIR.mkdir(exist_ok=True)

    # 1) Allocations LONG (benchmark doesn't belong here)
    alloc_long_parts = []

    def add_strategy(strategy_name, w_col, usd_col, sh_col):
        tmp = allocations_df_base[["Ticker", "Company", "LatestPrice"]].copy()
        tmp["Strategy"] = strategy_name
        tmp["IsBenchmark"] = False
        tmp["Weight"] = allocations_df_base[w_col].astype(float).values
        tmp["DollarAlloc"] = allocations_df_base[usd_col].astype(float).values
        tmp["Shares"] = allocations_df_base[sh_col].astype(float).values
        alloc_long_parts.append(tmp)

    add_strategy("MaxSharpe", "MaxSharpeWeight", "MaxSharpe$", "MaxSharpeShares")
    add_strategy("MinVol", "MinVolWeight", "MinVol$", "MinVolShares")
    add_strategy("MaxReturn", "MaxReturnWeight", "MaxReturn$", "MaxReturnShares")
    add_strategy("Equal", "EqualWeight", "EqualWeight$", "EqualWeightShares")

    allocations_long = pd.concat(alloc_long_parts, ignore_index=True)
    allocations_long.to_csv(PBI_OUT_DIR / "pb_allocations_long.csv", index=False)

    # 2) Backtest equity curves (constant weights) + BENCHMARK
    daily_ret_max = backtest_portfolio(returns, max_sharpe_w)
    daily_ret_min = backtest_portfolio(returns, min_vol_w)
    daily_ret_ret = backtest_portfolio(returns, max_return_w)
    daily_ret_eq = backtest_portfolio(returns, equal_weights)

    bt = pd.DataFrame({
        "Date": returns.index,
        "MaxSharpe": daily_ret_max.values,
        "MinVol": daily_ret_min.values,
        "MaxReturn": daily_ret_ret.values,
        "Equal": daily_ret_eq.values,
        BENCHMARK_NAME: bench_returns.values
    })

    bt_long = bt.melt(id_vars="Date", var_name="Strategy", value_name="DailyReturn")
    bt_long["IsBenchmark"] = bt_long["Strategy"].eq(BENCHMARK_NAME)
    bt_long["EquityCurve"] = bt_long.groupby("Strategy")["DailyReturn"].transform(lambda s: (1 + s).cumprod())
    bt_long["EquityCurve_$"] = bt_long["EquityCurve"] * amount_of_money
    bt_long.to_csv(PBI_OUT_DIR / "pb_portfolio_timeseries.csv", index=False)

    # 3) Metrics table (realised) + BENCHMARK
    m_max = perf_metrics_from_daily_returns(daily_ret_max, RISK_FREE_RATE)
    m_min = perf_metrics_from_daily_returns(daily_ret_min, RISK_FREE_RATE)
    m_ret = perf_metrics_from_daily_returns(daily_ret_ret, RISK_FREE_RATE)
    m_eq = perf_metrics_from_daily_returns(daily_ret_eq, RISK_FREE_RATE)
    m_bm = perf_metrics_from_daily_returns(bench_returns, RISK_FREE_RATE)

    metrics_df = pd.DataFrame([
        {"Strategy": "MaxSharpe", "IsBenchmark": False, **m_max},
        {"Strategy": "MinVol", "IsBenchmark": False, **m_min},
        {"Strategy": "MaxReturn", "IsBenchmark": False, **m_ret},
        {"Strategy": "Equal", "IsBenchmark": False, **m_eq},
        {"Strategy": BENCHMARK_NAME, "IsBenchmark": True, **m_bm},
    ])
    metrics_df.to_csv(PBI_OUT_DIR / "pb_metrics.csv", index=False)

    # 4) Efficient frontier points (optional Power BI scatter)
    frontier_points = pd.DataFrame({
        "PortfolioID": np.arange(len(vol_array)),
        "Volatility": vol_array,
        "ExpectedReturn": returns_array,
        "Sharpe": sharpe_array
    })
    frontier_points.to_csv(PBI_OUT_DIR / "pb_frontier_points.csv", index=False)

    # 5) Ticker map (optional)
    ticker_map = pd.DataFrame({"Ticker": final_tickers, "Company": company_names})
    ticker_map.to_csv(PBI_OUT_DIR / "pb_ticker_map.csv", index=False)

    print(f"\nPower BI exports saved to: {PBI_OUT_DIR.resolve()}")
    print("Files:")
    print("- pb_allocations_long.csv")
    print("- pb_portfolio_timeseries.csv (includes Benchmark)")
    print("- pb_metrics.csv (includes Benchmark)")
    print("- pb_frontier_points.csv")
    print("- pb_ticker_map.csv")

    # =========================================================
    # PLOTS + SAVE
    # =========================================================
    plt.figure(figsize=(10, 7))
    plt.scatter(vol_array, returns_array, c=sharpe_array, cmap="viridis", s=10, alpha=0.6)
    plt.colorbar(label="Sharpe Ratio")

    plt.scatter(max_sharpe_vol, max_sharpe_ret, s=250, marker="*", color="red", label="Max Sharpe")
    plt.scatter(min_vol_vol, min_vol_ret, s=250, marker="*", color="blue", label="Min Volatility")
    plt.scatter(max_return_vol, max_return_ret, s=250, marker="*", color="green", label="Max Return")

    plt.xlabel("Volatility")
    plt.ylabel("Expected Return")
    plt.title("Efficient Frontier")
    plt.legend()
    plt.grid(True)
    plt.savefig("efficient_frontier.png", dpi=300, bbox_inches="tight")
    plt.close()

    print("\nSaved: efficient_frontier.png")

    # Pie charts
    plt.figure(figsize=(8, 8))
    plt.pie(max_sharpe_w, labels=final_tickers, autopct="%1.1f%%")
    plt.title("Max Sharpe Portfolio")
    plt.savefig("max_sharpe_allocation.png", dpi=300, bbox_inches="tight")
    plt.close()

    plt.figure(figsize=(8, 8))
    plt.pie(min_vol_w, labels=final_tickers, autopct="%1.1f%%")
    plt.title("Minimum Volatility Portfolio")
    plt.savefig("min_vol_allocation.png", dpi=300, bbox_inches="tight")
    plt.close()

    plt.figure(figsize=(8, 8))
    plt.pie(max_return_w, labels=final_tickers, autopct="%1.1f%%")
    plt.title("Maximum Return Portfolio")
    plt.savefig("max_return_allocation.png", dpi=300, bbox_inches="tight")
    plt.close()

    plt.figure(figsize=(8, 8))
    plt.pie(equal_weights, labels=final_tickers, autopct="%1.1f%%")
    plt.title("Equal Weight Portfolio")
    plt.savefig("equal_weight_allocation.png", dpi=300, bbox_inches="tight")
    plt.close()

    print("Saved all pie charts.")


if __name__ == "__main__":
    main()


What is the total investment amount: $4500000


[*********************100%***********************]  51 of 51 completed



Ticker → Company mapping:
ACU.AX = Acumentis Group Limited
AGL.AX = AGL Energy Limited
AHE.AX = Adheris Health Limited
AIQ.AX = Alternative Investment Trust
AMS.AX = Atomos Limited
ANZ.AX = ANZ Group Holdings Limited
APW.AX = Aims Property Securities Fund
AQZ.AX = Alliance Aviation Services Limited
AVD.AX = AVADA Group Limited
BFL.AX = BSP Financial Group Limited
BHP.AX = BHP Group Limited
BNL.AX = Blue Star Helium Limited
BPH.AX = BPH Energy Limited
CAN.AX = Cann Group Limited
CCG.AX = Comms Group Limited
CLG.AX = Close the Loop Ltd
CLZ.AX = Classic Minerals Limited
COL.AX = Coles Group Limited
DAI.AX = Decidr AI Industries Ltd
DGT.AX = DIGICO INF STAPLED [DGT]
DTI.AX = DTI Group Limited
EDV.AX = Endeavour Group Limited
ENN.AX = Elanor Investors Group
EVN.AX = Evolution Mining Limited
EWC.AX = Energy World Corporation Ltd
FCG.AX = Freedom Care Group Holdings Limited
GTN.AX = GTN Limited
GYG.AX = Guzman y Gomez (Holdings) Ltd
ILA.AX = Island Pharmaceuticals Limited
INV.AX = InvestSMAR

[*********************100%***********************]  1 of 1 completed



CSV saved: Portfolio_Optimisation_Results.csv

Power BI exports saved to: C:\Users\jed1n\Documents\portfolio_optimizer\powerbi_exports
Files:
- pb_allocations_long.csv
- pb_portfolio_timeseries.csv (includes Benchmark)
- pb_metrics.csv (includes Benchmark)
- pb_frontier_points.csv
- pb_ticker_map.csv

Saved: efficient_frontier.png
Saved all pie charts.
